In [25]:
##package needed to do the exercise
import pandas as pd #package for reading and using dataset
import numpy as np #package needed for inserting value in dataset
from sklearn.metrics import roc_auc_score #needed for determing AUC score
import time as t
begin = t.perf_counter()

In [26]:
df=pd.read_csv('C:/Users/s151675/Documents/Jaar_master_2/Kwartiel 3,4/Master project/Data aangepast/data_for_prediction/data+prediction_final/data_complete_final.csv', delimiter = ';') #read csv file

In [27]:
df #visualize dataset

,Case ID,Activity,Complete Timestamp,Variant,Variant index,Specialismecode,Age,Gender,Triage color,Transport,...,Labaanvraag_SEH,Rad_to_ext,Consult_SEH,Opname,Bigger4,ArrivelDate,ArrivelTime,DepartDate,DepartTime,Unnamed: 24
0,155011,Medicatieverificatie SEH,2019-01-01 00:35:00.000,Variant 2252,2252,CHI,26,Man,Geel,Ambulance,...,1,0,1,1,0,2019-01-01 00:00:00.000,"0,024305556",2019-01-01 00:00:00.000,"0,139583333","0,115277778"
1,155013,Radiologie,2019-01-01 00:37:00.000,Variant 2253,2253,NEU,73,Man,Oranje,Ambulance,...,0,0,0,1,1,2019-01-01 00:00:00.000,"0,025694444",2019-01-01 00:00:00.000,"0,227083333","0,201388889"
2,155020,Medicatieverificatie SEH,2019-01-01 01:29:00.000,Variant 4,4,CHI,32,Man,Groen,Eigen vervoer,...,0,0,0,0,0,2019-01-01 00:00:00.000,"0,061805556",2019-01-01 00:00:00.000,"0,086111111","0,024305556"
3,155021,Medicatieverificatie SEH,2019-01-01 01:26:00.000,Variant 2254,2254,INT,34,Man,Oranje,Ambulance,...,1,0,0,1,0,2019-01-01 00:00:00.000,"0,059722222",2019-01-01 00:00:00.000,"0,202083333","0,142361111"
4,155027,Medicatieverificatie SEH,2019-01-01 01:54:00.000,Variant 1252,1252,CAR,21,Vrouw,Geel,Eigen vervoer,...,1,0,0,0,0,2019-01-01 00:00:00.000,"0,079166667",2019-01-01 00:00:00.000,"0,225694444","0,146527778"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50078,243220,Pijnscore herhalen,2020-08-02 21:36:00.000,Variant 455,455,CHI,60,Vrouw,Groen,Eigen vervoer,...,0,0,0,0,0,2020-08-02 00:00:00.000,"0,85625",2020-08-02 00:00:00.000,"0,909027778","0,052777778"
50079,243221,Pijnscore herhalen,2020-08-02 21:36:00.000,Variant 36877,36877,CHI,27,Man,Geel,Eigen vervoer,...,1,1,0,0,0,2020-08-02 00:00:00.000,"0,895833333",2020-08-02 00:00:00.000,"0,994444444","0,098611111"
50080,243223,Beleidsorder SEH,2020-08-02 23:20:00.000,Variant 1587,1587,NEU,77,Vrouw,Geel,Ambulance,...,1,0,0,0,0,2020-08-02 00:00:00.000,"0,972222222",2020-08-03 00:00:00.000,"0,102083333","0,129861111"
50081,243224,Beleidsorder SEH-VPL,2020-08-02 22:48:00.000,Variant 36878,36878,CAR,63,Vrouw,Geel,Eigen vervoer,...,1,0,0,0,0,2020-08-02 00:00:00.000,"0,941666667",2020-08-03 00:00:00.000,"0,045833333","0,104166667"


In [28]:
##packages needed for using decision tree classifier and splitting dataset
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [29]:
#Considered one-hot-encoding in our data, see report.
spec_dummies = pd.get_dummies(df.Specialismecode) #OneHotencoder used for column job
transport_dummies = pd.get_dummies(df.Transport) #OneHotencoder used for column marital

##replace values from categorical to nummerical 
df = df.replace('Vrouw', 0)
df = df.replace('Man', 1)
##replace values from categorical to nummerical (with order)
df = df.replace('Blauw',0) 
df = df.replace('Groen', 1)
df = df.replace('Geel', 2)
df = df.replace('Oranje', 3)
df = df.replace('Rood', 4)
##replace null values with -1
df = df.fillna(-1)

# df = pd.concat([df.iloc[:,0],spec_dummies,df.iloc[:,6:9], transport_dummies, df.iloc[:,11:15], practice_dummies, df.iloc[:,16:24]], axis=1) #Dataset with new numerical columns(job and marital)
df = pd.concat([df.iloc[:,0],spec_dummies,df.iloc[:,6:9], transport_dummies, df.iloc[:,11:12],df.iloc[:,12:15], df.iloc[:,19]], axis=1) #Dataset with new numerical columns(job and marital)
# iterating the columns
df



,Case ID,ANE,AVG,CAR,CHI,CTC,DER,GYN,HEM,ICA,...,Ambulance (Inzet heli-MMT),Ambulance (inzet grond-MMT),Anders,Eigen vervoer,Trauma helikopter,History_bin,nr_patients_same_spec,nr_patients_total,Presentation_time,Bigger4
0,155011,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,155013,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
2,155020,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,1,3,0,0
3,155021,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
4,155027,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50078,243220,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,1,4,13,8,0
50079,243221,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,1,6,16,8,0
50080,243223,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,9,9,0
50081,243224,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,1,11,9,0


In [30]:
##define input variables(X) and output variable(Y)
feature_cols2 = df.columns
feature_cols = feature_cols2[1:df.shape[1]-1]
# feature_cols_check = feature_cols2[1:df.shape[1]-1]
feature_cols_x = feature_cols2[1:df.shape[1]-1] # all columns excluding 'y' and case_ID
feature_cols_y = feature_cols2[df.shape[1]-1:df.shape[1]] 
# feature_cols = ['to_extern', 'INT', 'CHI', 'Age', 'registering_time', 'MDL', 'ORT', 'History_bin', 'CAR', 'Triage color', 'NEU','Lab']#take relevant features to speed up process
X = df[feature_cols_x] #input variables
X_ana = df[feature_cols]
# X_ana_big = df[feature_cols_check]
y = df[feature_cols_y]# Target variable
print(X)
print(y)


       ANE  AVG  CAR  CHI  CTC  DER  GYN  HEM  ICA  INT  ...  Ambulance  \
0        0    0    0    1    0    0    0    0    0    0  ...          1   
1        0    0    0    0    0    0    0    0    0    0  ...          1   
2        0    0    0    1    0    0    0    0    0    0  ...          0   
3        0    0    0    0    0    0    0    0    0    1  ...          1   
4        0    0    1    0    0    0    0    0    0    0  ...          0   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...        ...   
50078    0    0    0    1    0    0    0    0    0    0  ...          0   
50079    0    0    0    1    0    0    0    0    0    0  ...          0   
50080    0    0    0    0    0    0    0    0    0    0  ...          1   
50081    0    0    1    0    0    0    0    0    0    0  ...          0   
50082    0    0    0    0    0    0    0    0    0    0  ...          1   

       Ambulance (Inzet heli-MMT)  Ambulance (inzet grond-MMT)  Anders  \
0                        

In [31]:
# Split dataset into training set and test set
X_train_t, X_test_t, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # divide dataset into 70% training and 30% test
X_train = X_train_t.iloc[:,0:40]
X_test = X_test_t.iloc[:,0:40]
y_train_lab = y_train.iloc[:,0]
# y_train_rad_to_ext = y_train.iloc[:,1]
# y_train_consult = y_train.iloc[:,2]
# y_train_opname = y_train.iloc[:,3]
y_test_lab = y_test.iloc[:,0]
# y_test_rad_to_ext = y_test.iloc[:,1]
# y_test_consult = y_test.iloc[:,2]
# y_test_opname = y_test.iloc[:,3]
X_number = X_test_t.iloc[:,0] #take caseID from test set to compare with actual value in later phase
###dataset needed for making predictions with already predicted values
# X_train_big = pd.concat([X_train_t.iloc[:,1:55],y_train.iloc[:,0:4]], axis=1)
##choose between these test_data set, where perf is with actual values and other is with predicted values
# X_test_big = X_test_t.iloc[:,1:55] #make this out of prediction
# X_test_big_perf = pd.concat([X_test_t.iloc[:,1:40],y_test.iloc[:,0:4]], axis=1) #to compare with existing values of columns
print(X_test)
print(y_test_lab)
# y_train_big = y_train.iloc[:,-1]
# y_test_big = y_test.iloc[:,-1]

       ANE  AVG  CAR  CHI  CTC  DER  GYN  HEM  ICA  INT  ...  Ambulance  \
49240    0    0    0    1    0    0    0    0    0    0  ...          0   
9336     0    0    0    0    0    0    0    0    0    0  ...          0   
25404    0    0    0    0    0    0    0    0    0    0  ...          0   
49563    0    0    0    1    0    0    0    0    0    0  ...          1   
20947    0    0    0    1    0    0    0    0    0    0  ...          1   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...        ...   
34566    0    0    0    0    0    0    0    0    0    0  ...          0   
41887    0    0    0    0    0    0    0    0    0    0  ...          1   
822      0    0    0    0    0    0    0    0    0    0  ...          0   
9942     0    0    0    1    0    0    0    0    0    0  ...          0   
37540    0    0    1    0    0    0    0    0    0    0  ...          1   

       Ambulance (Inzet heli-MMT)  Ambulance (inzet grond-MMT)  Anders  \
49240                    

In [32]:
#Function made to measure the confusing matrix
def perf_measure(y_actual, y_hat):
    ## start with zero values
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    count = 0 
    for i in range(len(y_hat)): #for all values in the column y_test
        count += 1 #count the total number of rows
        if y_actual[i]==1 and y_hat[i]==1:#True positive
            TP += 1
        if y_hat[i]==1 and y_actual[i]==0:#False positive
            FP += 1
        if y_actual[i]==0 and y_hat[i]==0:#True negative
            TN += 1
        if y_hat[i]==0 and y_actual[i]==1:#False negative
            FN += 1

    return(TP/count, FP/count, TN/count, FN/count, count)#return all relevant values in fractions

In [33]:
def RandomForest(Number_leaves, X_train, y_train, X_test):
    clf = RandomForestClassifier(n_estimators = 100, min_samples_leaf=Number_leaves,class_weight = "balanced") #define decision tree with selected variable
    clf = clf.fit(X_train,y_train) #train the decision tree on training set
    y_pred = clf.predict(X_test) #predict the values of X_test
    
    return(clf, y_pred)


In [34]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [35]:
def predict_value(y_train, y_test, X_train, X_test, X_ana, feature_cols2, negative):
    ##used for determing best values
    max_score = 0
    max_AUC = 0
    best_leaves = 0
    y_test = np.array(y_test)
    clf_final, y_pred = RandomForest(2, X_train, y_train, X_test)
    ##loop over values from 2 to 100 for selecting best parameter
    for i in range(2,200):
        clf, y_pred = RandomForest(i,X_train, y_train, X_test)
    #     AUC_score = roc_auc_score(y_test,y_pred)
    #     print(accuracy_score(y_test_lab, y_pred))
        TPRate,FPRate,TNRate, FNRate, count = perf_measure(y_test, y_pred) #call confusing matrix function to determine these values in matrix
        if(negative==False):
            accuracy = (TPRate+TNRate)/(TPRate+TNRate+FPRate+FNRate)
            print("accuracy :", accuracy," for num of leaves =", i)
        else:
            accuracy = TNRate/(TNRate+FNRate)
            print("Negative prediction value :", accuracy," for num of leaves =", i)
        ##function used for updating best score if one score exceed previously best scores
        
        if accuracy>=max_score:
            max_score = accuracy
            best_leaves = i
            clf_final = clf
    
    clf, y_pred = RandomForest(best_leaves,X_train, y_train, X_test) #Best decision tree based on highest accuracy 
    TPRate,FPRate,TNRate, FNRate, count = perf_measure(y_test, y_pred) #call confusing matrix function to determine these values in matrix
    p = (((TNRate+FPRate)*count)/count*((TNRate+FNRate)*count)/count) #measure p value for best model based on recall & AUC score
    kappa_score = (metrics.accuracy_score(y_test, y_pred)-p)/(1-p) #measure Kappa score for best model based on recall & AUC score
    AUC_score = roc_auc_score(y_test,y_pred) #measure AUC score for best model based on recall & AUC score
    ##measure Accuracy score for best model based on Recall & AUC score
    accuracy = (TPRate+TNRate)/(TPRate+TNRate+FPRate+FNRate)

    ###feature importance start
    forest = ExtraTreesClassifier(n_estimators=100,min_samples_leaf=best_leaves, class_weight = "balanced")
    forest = forest.fit(X_train,y_train)

    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
    indices = np.argsort(importances)[::-1]
    ##print all scores for best model based on recall & AUC score
    print("best number of leaves = ", best_leaves, " with score = ", max_score, " and accuracy = ", accuracy)
    print("kappa score =", kappa_score)
    print("AUC score =", AUC_score)
    print(confusion_matrix(y_test,y_pred))
    # Print the feature ranking
    print("Feature ranking:")
    for f in range(0,X_ana.shape[1]):
        print("%d. %s (%f)" % (f + 1, feature_cols2[indices[f]], importances[indices[f]]))
    return(clf_final)


In [36]:
forest_lab = predict_value(y_train_lab, y_test_lab, X_train, X_test, X_ana, feature_cols, True)
eind = t.perf_counter()
print("number of seconds =", eind-begin)
# forest_rad_to_ext = predict_value(y_train_rad_to_ext, y_test_rad_to_ext, X_train, X_test, X_ana, feature_cols, False)
# forest_consult = predict_value(y_train_consult, y_test_consult, X_train, X_test, X_ana, feature_cols, False)
# forest_opname = predict_value(y_train_opname, y_test_opname, X_train, X_test, X_ana, feature_cols, False)

# lab_prediction = forest_lab.predict(X_test)
# rad_to_ext_prediction = forest_rad_to_ext.predict(X_test)
# consult_prediction = forest_consult.predict(X_test)
# opname_prediction = forest_opname.predict(X_test)
# X_test_big.insert(54,"labaanvraag", lab_prediction)
# X_test_big.insert(55,"rad_to_ext", rad_to_ext_prediction)
# X_test_big.insert(56,"consult", consult_prediction)
# X_test_big.insert(57,"opname", opname_prediction)
# forest_big4 = predict_value(y_train_big, y_test_big, X_train_big, X_test_big, X_ana_big, feature_cols_check, True) ##have to change X_train last features
# big4_prediction = forest_big4.predict(X_test_big)
# X_test_big.insert(58,"bigger than 4", big4_prediction)

# dataframeafter = pd.concat([X_number,X_test_big.iloc[:,0:57]],axis=1) ##add bigger4 prediction
# dataframeafter.to_csv('C:/Users/s151675/Documents/Jaar_master_2/Kwartiel 3,4/Master project/Data aangepast/Data_for_prediction/final_testgroup_prediction.csv')

Negative prediction value : 0.7578823741982113  for num of leaves = 2
Negative prediction value : 0.7729111176876788  for num of leaves = 3
Negative prediction value : 0.7799588900308324  for num of leaves = 4
Negative prediction value : 0.7833173537871525  for num of leaves = 5
Negative prediction value : 0.7875256285745117  for num of leaves = 6
Negative prediction value : 0.790064985130521  for num of leaves = 7
Negative prediction value : 0.7919830281375615  for num of leaves = 8
Negative prediction value : 0.7919378782688136  for num of leaves = 9
Negative prediction value : 0.7943850875198909  for num of leaves = 10
Negative prediction value : 0.7951017592273198  for num of leaves = 11
Negative prediction value : 0.796110919341848  for num of leaves = 12
Negative prediction value : 0.7970211291998615  for num of leaves = 13
Negative prediction value : 0.796590378327884  for num of leaves = 14
Negative prediction value : 0.798514501296864  for num of leaves = 15
Negative predictio

Negative prediction value : 0.8018051434223541  for num of leaves = 118
Negative prediction value : 0.7999019127023052  for num of leaves = 119
Negative prediction value : 0.8001729035445228  for num of leaves = 120
Negative prediction value : 0.7983880967141971  for num of leaves = 121
Negative prediction value : 0.7991082486995293  for num of leaves = 122
Negative prediction value : 0.8009111056390052  for num of leaves = 123
Negative prediction value : 0.7983162518301611  for num of leaves = 124
Negative prediction value : 0.7993630573248407  for num of leaves = 125
Negative prediction value : 0.7988923076923077  for num of leaves = 126
Negative prediction value : 0.8003940401428395  for num of leaves = 127
Negative prediction value : 0.7983224373997779  for num of leaves = 128
Negative prediction value : 0.7994388190801514  for num of leaves = 129
Negative prediction value : 0.7983751846381093  for num of leaves = 130
Negative prediction value : 0.7989894010352476  for num of leave